In [1]:
import tensorflow as tf
import keras
import math
import numpy as np
import json

In [5]:
# create base vgg model
base_model = tf.keras.applications.VGG16(
    input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(units=7, activation="softmax")  


inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = global_average_layer(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

# construct the model
## model = tf.keras.models.Sequential([
## 
##     # Convolutional Block 1
##     tf.keras.layers.Conv2D(input_shape=(48, 48, 1),
##         filters=4, kernel_size=(4, 4), strides=(1, 1), padding="same", activation="relu"),
##     tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
## 
##     # Convolutional Block 2
##     tf.keras.layers.Conv2D(filters=8, kernel_size=(3, 3), strides=(2, 2), padding="same", activation="relu"),
##     tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
## 
##     # convolutional Block 3
##     tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), strides=(2, 2), padding="same", activation="relu"),
##     tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
## 
##     # Convolutional Block 4
##     tf.keras.layers.Conv2D(filters=32, kernel_size=(2, 2), strides=(2, 2), padding="same", activation="relu"),
## 
##     tf.keras.layers.Flatten(),
##     tf.keras.layers.Dense(units=160,activation="relu"),
##     tf.keras.layers.Dropout(0.5),
##     tf.keras.layers.Dense(units=7, activation="softmax")    
##   
## ])


### # display the model summary
model.summary()
### 
### # plot the model architecture
### tf.keras.utils.plot_model(
###     model, to_file='./images/architecture.png', show_shapes=True, show_dtype=False,
###     show_layer_names=True, rankdir='TB', expand_nested=False, dpi=150,
###     layer_range=None, show_layer_activations=False
### )

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d_2   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_6 (Dense)             (None, 7)                 3591      
                                                                 
Total params: 14,718,279
Trainable params: 3,591
Non-trainable params: 14,714,688
_________________________________________________________________


In [3]:
epochs = 75
initial_lrate = 0.01

def decay(epoch, steps=100):
    initial_lrate = 0.01
    drop = 0.98
    epochs_drop = 10
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

sgd = tf.keras.optimizers.SGD(learning_rate=initial_lrate, momentum=0.9, nesterov=False)

lr_sc = tf.keras.callbacks.LearningRateScheduler(decay, verbose=1)

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=sgd, metrics=['accuracy'])

In [ ]:
ds_path = r'/Users/heathsmith/repos/github/neuromorphic-computing/datasets/ck_plus_48.npz'

with np.load(ds_path) as data:
    images = data['images']
    labels = data['labels']
    classes = data['classes']
print(np.shape(images), np.shape(labels))

# constants
BATCH_SIZE = 16
_TRAIN = .80
_VAL = .10
_TEST = .10
SEED = 123

# create a dataset object and shuffle
dataset = tf.data.Dataset.from_tensor_slices((images, labels))
dataset = dataset.shuffle(len(images) * BATCH_SIZE, seed=SEED)
shuffle_images = []
shuffle_labels = []
for im, l in dataset.as_numpy_iterator():
    shuffle_images.append(im)
    shuffle_labels.append(l)
shuffle_images = np.asarray(shuffle_images)
shuffle_labels = np.asarray(shuffle_labels)
print(np.shape(shuffle_images))
print(len(dataset))

# create the slicing indices
TRAIN = round(len(dataset) * _TRAIN)
VAL = TRAIN + round(len(dataset) * _VAL)
TEST = VAL + round(len(dataset) * _TEST)
print(TRAIN, VAL, TEST)

x_train = list(map(lambda x: np.expand_dims(x, axis=2), shuffle_images[0:TRAIN]))
x_train = list(map(lambda x: np.array(x/255.0).astype(np.float32), x_train))
train_ds =  tf.data.Dataset.from_tensor_slices((x_train, shuffle_labels[0:TRAIN]))
train_ds = train_ds.cache()
train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
train_ds = train_ds.batch(batch_size=BATCH_SIZE, num_parallel_calls=tf.data.AUTOTUNE, drop_remainder=False)
print(np.shape(x_train))

x_val = list(map(lambda x: np.expand_dims(x, axis=2), shuffle_images[TRAIN:VAL]))
x_val = list(map(lambda x: np.array(x/255.0).astype(np.float32), x_val))
val_ds = tf.data.Dataset.from_tensor_slices((x_val, shuffle_labels[TRAIN:VAL]))
val_ds = val_ds.cache()
val_ds = val_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.batch(batch_size=BATCH_SIZE, num_parallel_calls=tf.data.AUTOTUNE, drop_remainder=False)
print(np.shape(shuffle_images[TRAIN:VAL]))

x_test = list(map(lambda x: np.expand_dims(x, axis=2), shuffle_images[VAL:TEST]))
x_test = list(map(lambda x: np.array(x/255.0).astype(np.float32), x_test))
test_ds = tf.data.Dataset.from_tensor_slices((x_test, shuffle_labels[VAL:TEST]))
test_ds = test_ds.cache()
test_ds = test_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.batch(batch_size=BATCH_SIZE,num_parallel_calls=tf.data.AUTOTUNE,drop_remainder=False)

In [ ]:

# add checkpoint callback
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    './inception.h5',
    monitor='val_accuracy',
    verbose=0,
    save_best=True,
    save_weights_only=False,
    mode='max'
)

history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, batch_size=BATCH_SIZE, callbacks=[lr_sc, checkpoint], verbose=1)

In [ ]:
#json.dump(history.history, open('./inception_history.json', 'w'))
print(history.history)

In [ ]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

# plot the results
epochs_range = np.arange(len(history.history['accuracy']))
fig = make_subplots(rows=1, cols=2, subplot_titles=('Accuracy', 'Loss'))

fig.add_trace(
    go.Scatter(
        x=epochs_range, y=history.history['accuracy'],
        name='Training', legendgroup='accuracy', mode='lines',  line=dict(color='blue'),
        legendgrouptitle_text='Accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=epochs_range, y=history.history['val_accuracy'], mode='lines',  line=dict(color='green'),
    name='Validation', legendgroup='accuracy'), row=1, col=1)

fig.add_trace(
    go.Scatter(x=epochs_range, y=history.history['loss'], line=dict(color='blue'),
    mode='lines', showlegend=False), row=1, col=2)
fig.add_trace(
    go.Scatter(x=epochs_range, y=history.history['val_loss'], line=dict(color='green'),
    mode='lines', showlegend=False), row=1, col=2)


fig.update_layout(
    template='plotly_dark',
    title="Inception - CK+ Dataset",
    title_x=0.5
    ## xaxis_title="Epochs"
    ## yaxis_title="LR Value",)
)

# save and display the plot image
pio.write_image(fig, "./history.png", scale=6, width=1000, height=600)
fig.show()

In [ ]:
# evaluate the model
loss, acc = model.evaluate(test_ds, verbose=2)